In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import pandas as pd
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import glob
import time

In [11]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : os.getcwd()}
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
service = ChromeService(executable_path=ChromeDriverManager().install())

C:\Users\ebroh\AppData\Local\Temp\ipykernel_103352\846415680.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
C:\Users\ebroh\AppData\Local\Temp\ipykernel_103352\846415680.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [3]:
def openBudgetSite(driver, school_code, fiscal_year, budgetOrAllocation = 'budget'):
    if (budgetOrAllocation == 'allocation'):
        driver.get("https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/galaxy/galaxyallocation/default.aspx")

    elif (budgetOrAllocation == 'budget'):
        driver.get("https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/galaxy/galaxybudgetsummaryto/default.aspx")
        
    else:
        raise ValueError('Budget or Allocation value must be either "budget" or "allocation"')
                         
    # get element 
    element = driver.find_element(By.XPATH,'//*[@id="School_Code"]')

    # send keys 
    element.send_keys(school_code)

    x = driver.find_element(By.XPATH,'//*[@id="Fiscal_Year"]')
    drop=Select(x)

    # select by visible text
    drop.select_by_visible_text(fiscal_year)

    driver.find_element(By.XPATH,'//*[@id="Enter"]').click()
    

    try:
        element = driver.find_element(By.XPATH,'//*[@id="message"]/div[1]/div[3]/h2/a')
        element.text.index(school_code)
        return True

    except NoSuchElementException:
        print('No ' + budgetOrAllocation + ' data could be found for school code ' + school_code + ' in the year ' + year)
        return False
        
    except ValueError:
        print('Given School code and school code for retrieved data do not match')
        return False


def allocationPageScraper(driver, school_code, fiscal_year):
    list_of_dfs = pd.read_html(driver.page_source)
    final_df = pd.DataFrame(columns=['location_code', 'fiscal_year', 'allocation_category', 'amount'])

    for df in list_of_dfs:
        df = df.rename(columns={df.columns[0]: 'allocation_category', df.columns[1]: 'amount'})
        df = df[df["allocation_category"].str.contains("total", case = False)==False]
        df['location_code'] = school_code
        df['fiscal_year'] = fiscal_year
        final_df = pd.concat([final_df, df])
    
    return final_df
    
    
def budgetPageScraper(driver, school_code, fiscal_year):
    section_titles = driver.find_elements(By.CLASS_NAME , 'TO_Section')
    list_of_dfs = pd.read_html(driver.page_source)
    
    final_df = pd.DataFrame(columns=['location_code', 'fiscal_year', 'budget_category', 'budget_assignment', 'num_positions', 'service_type', 'amount'])

    for i,df in enumerate(list_of_dfs):
        if (len(df.columns) == 2):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'amount'})
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            df['num_positions'] = None
            df['service_type'] = None
            final_df = pd.concat([final_df, df])
        
        elif (len(df.columns) == 3):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'num_positions', df.columns[2]: 'amount'})
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            df['service_type'] = None
            final_df = pd.concat([final_df, df])

        elif (len(df.columns) == 4):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'service_type', df.columns[2]: 'num_positions', df.columns[3]: 'amount'})
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df = df[df["service_type"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            final_df = pd.concat([final_df, df])
            
    return final_df

In [4]:
if (openBudgetSite(driver, 'M125', '2022', 'budget')):
    df = budgetPageScraper(driver, 'M125', '2022')

In [5]:
df

,location_code,fiscal_year,budget_category,budget_assignment,num_positions,service_type,amount
0,M125,2022,Leadership,HEAD OF SCHOOL,1.0,None,"$ 84,471"
1,M125,2022,Leadership,AP - SUPERVISION,1.0,None,"$ 121,384"
0,M125,2022,Coordinator/Supervisor/Dean,PARENT COORDINATOR,1.0,None,"$ 53,241"
0,M125,2022,Secretary,School Secretary,1.0,None,"$ 76,956"
0,M125,2022,Classroom Teacher,Pre K,2.0,WHOLE CLASS,"$ 178,549"
1,M125,2022,Classroom Teacher,K,2.0,WHOLE CLASS,"$ 207,632"
2,M125,2022,Classroom Teacher,"K,1",1.0,SELF-CONTAINED SP ED,"$ 97,194"
3,M125,2022,Classroom Teacher,1,2.0,WHOLE CLASS,"$ 198,553"
4,M125,2022,Classroom Teacher,2,2.0,WHOLE CLASS,"$ 194,824"
5,M125,2022,Classroom Teacher,3,2.0,WHOLE CLASS,"$ 231,099"


In [6]:
school_data = pd.read_csv(r'C:\Users\ebroh\Downloads\2019_-_2020_School_Locations (2).csv')

In [7]:
school_data['location_code'].unique()

array(['M010', 'M030', 'M036', 'M046', 'M092', 'M123', 'M125', 'M129',
       'M133', 'M148', 'M154', 'M157', 'M161', 'M172', 'M175', 'M194',
       'M195', 'M197', 'M200', 'M275', 'M285', 'M286', 'M302', 'M304',
       'M317', 'M318', 'M344', 'M362', 'M367', 'M369', 'M410', 'M436',
       'M469', 'M499', 'M514', 'M517', 'M532', 'M670', 'M685', 'M692'],
      dtype=object)

In [13]:
budget_data = pd.DataFrame()
allocation_data = pd.DataFrame()
years = ['2018', '2019', '2020', '2021', '2022']

for school in school_data['location_code'].unique():
    print(school)
    for year in years:
        if (openBudgetSite(driver, school, year, 'budget')):
            print(year +  ' Budget Data Opened')
            budget_data = pd.concat([budgetPageScraper(driver, school, year), budget_data])
        if (openBudgetSite(driver, school, year, 'allocation')):
            print(year +  ' Allocation Data Opened')
            allocation_data = pd.concat([allocationPageScraper(driver, school, year), allocation_data])

M010
No budget data could be found for school code M010 in the year 2018
No allocation data could be found for school code M010 in the year 2018
No budget data could be found for school code M010 in the year 2019
No allocation data could be found for school code M010 in the year 2019
No budget data could be found for school code M010 in the year 2020
No allocation data could be found for school code M010 in the year 2020
No budget data could be found for school code M010 in the year 2021
No allocation data could be found for school code M010 in the year 2021
No budget data could be found for school code M010 in the year 2022
No allocation data could be found for school code M010 in the year 2022
M030
2018 Budget Data Opened
2018 Allocation Data Opened
2019 Budget Data Opened
2019 Allocation Data Opened
2020 Budget Data Opened
2020 Allocation Data Opened
2021 Budget Data Opened
2021 Allocation Data Opened
2022 Budget Data Opened
2022 Allocation Data Opened
M036
2018 Budget Data Opened
2

2018 Budget Data Opened
2018 Allocation Data Opened
2019 Budget Data Opened
2019 Allocation Data Opened
2020 Budget Data Opened
2020 Allocation Data Opened
2021 Budget Data Opened
2021 Allocation Data Opened
2022 Budget Data Opened
2022 Allocation Data Opened
M317
No budget data could be found for school code M317 in the year 2018
No allocation data could be found for school code M317 in the year 2018
No budget data could be found for school code M317 in the year 2019
No allocation data could be found for school code M317 in the year 2019
No budget data could be found for school code M317 in the year 2020
No allocation data could be found for school code M317 in the year 2020
No budget data could be found for school code M317 in the year 2021
No allocation data could be found for school code M317 in the year 2021
No budget data could be found for school code M317 in the year 2022
No allocation data could be found for school code M317 in the year 2022
M318
2018 Budget Data Opened
2018 A

In [14]:
budget_data

,location_code,fiscal_year,budget_category,budget_assignment,num_positions,service_type,amount
0,M692,2022,Leadership,HEAD OF SCHOOL,1.0,None,"$ 197,026"
1,M692,2022,Leadership,AP - ORGANIZATION,1.0,None,"$ 148,166"
0,M692,2022,Coordinator/Supervisor/Dean,PARENT COORDINATOR,1.0,None,"$ 43,968"
0,M692,2022,Secretary,School Secretary,2.0,None,"$ 149,545"
0,M692,2022,High School Departments,ENGLISH LANGUAGE ARTS,1.0,SUBJECT SPECIFIC,"$ 106,797"
...,...,...,...,...,...,...,...
12,M030,2018,OTPS,TEXTBOOKS,NaN,None,"$ 7,756"
13,M030,2018,OTPS,TRANSPORTATION OF PUPILS - CONTRACTUAL,NaN,None,"$ 2,765"
14,M030,2018,OTPS,TRANSPORTATION OF STAFF - NON-CONTRACT,NaN,None,"$ 1,412"
0,M030,2018,People Working Partial Year,People Working Partial Year,NaN,None,"$ 101,435"


In [15]:
allocation_data

,location_code,fiscal_year,allocation_category,amount
0,M692,2022,AIDP Attendance Shared,"$ 150,490"
1,M692,2022,ARPA Academic Recovery,"$ 112,682"
2,M692,2022,ARPA Academic Recovery Arts,"$ 28,171"
3,M692,2022,ARPA Academic Recovery SE After-School Instruc...,"$ 16,242"
4,M692,2022,ARPA Academic Recovery SE Counseling,"$ 2,223"
...,...,...,...,...
26,M030,2018,TL Summer in the City Shared,"$ 12,969"
27,M030,2018,TL Translation Services,"$ 1,027"
0,M030,2018,TL Vision for School Improvement,"$ 19,097"
1,M030,2018,Universal PreK Quality Full-Day,"$ 205,776"
